# Chapter 1. The World's Smallest Django Project

---

# TOC

## 1. Hello Django

### 1) Creating the View

### 2) The URL Patterns

### 3) The Settings

### 4) Running the Example

## 2. Improvements

### 1) WSGI Application

### 2) Additional Configuration

### 3) Reusable Template

---

## 1. Hello Django

While the **startproject** command is a useful tool, it is not required in order to start a Django project. You are free to lay out your project however you like based on what you want to do. For larger projects, developers benefit from the code organization provided by the startproject command. However, **the convenience of this command shouldn’t stop you from understanding what it does and why it is helpful.**
In this chapter we’ll lay out an example of how to create a simple project using Django’s basic building blocks.

This lightweight “Hello World” project will create a simple Django application **using a single-file approach.**

In this chapter, we’ll start by using a single ***hello.py*** file. This file will contain all of the code needed to run our Django project.

### 1) Creating the View

model-template-view (MTV) framework
- The view portion typically
    - inspects the incoming HTTP request
    - queries the necessary data to send to the presentation layer.
    - constructs the necessary data to send to the presentation layer.

**let’s create a simple way to execute a “Hello World” response.**

In [ ]:
from django.http import HttpResponse

def index(request):
    return HttpResponse('Hello World')

- this would typically be in a views.py file inside one of your apps.
- there is no requirement for views to live inside of apps.
- There is also no requirement that views live in a file called views.py.
- This is purely a matter of convention, but not a requirement on which to base our project’s structure.

### 2) The URL Patterns

- Django associates views with their URL by pairing a regular expression to match the URL and any callable arguments to the view.
- The following is an example from hello.py of how we make this connection.

In [ ]:
from django.conf.urls import url
from django.http import HttpResponse

def index(request):
    return HttpResponse('Hello World')

urlpatterns = (
    url(r'^$', index),
)

- Now this file combines both a typical views.py file and the root urls.py file.
- Again, it is worth noting that there is **no requirement for the URL patterns to be included in a urls.py file.**

### 3) The Settings

- **Django settings** detail everything from database and cache connections to internation‐ alization features and static and uploaded resources.
- For many developers just getting started, **the settings in Django are a major point of confusion.**
- While **recent releases have worked to trim down the default settings’ file length**, it can still be overwhelming.

In [ ]:
from django.conf import settings

settings.configure(
    DEBUG=True,
    SECRET_KEY='thisisthesecretkey',
    ROOT_URLCONF=__name__,
    MIDDLEWARE_CLASSES=(
        'django.middleware.common.CommonMiddleware',
        'django.middleware.csrf.CsrfViewMiddleware',
        'django.middleware.clickjacking.XFrameOptionsMiddleware',
    ),
)

from django.conf.urls import url
from django.http import HttpResponse

def index(request):
    return HttpResponse('Hello World')

urlpatterns = (
    url(r'^$', index),
)

- A **secret key** must be generated for the default session and cross-site request forgery (CSRF) protection.
- It is important for any production site to have a **random SECRET_KEY** that is kept private.
- https://docs.djangoproject.com/en/1.7/topics/signing

#### Cryptographic signing

The golden rule of Web application security is to never trust data from untrusted sources. **Cryptographically signed values** can be passed through an untrusted channel safe in the knowledge that any tampering will be detected.

**Django provides both a low-level API for signing values and a high-level API** for setting and reading signed cookies, one of the most common uses of signing in Web applications.

##### **Protecting the SECRET_KEY** ->  Securing signed data

When you create a new Django project using startproject, the **settings.py** file is generated automatically and gets a random **SECRET_KEY value**.

**This value is the key to securing signed data** – it is vital you keep this secure, or attackers could use it to generate their own signed values.

##### **Using the low-level API**

###### sign

In [ ]:
from django.core.signing import Signer
from django.core import signing

In [ ]:
signer = Signer()
value = signer.sign('My String')
print(value)

###### unsign

In [ ]:
original = signer.unsign(value)
print(original)

###### tampering

In [ ]:
value += 'm'
try:
    original = signer.unsign(value)
except signing.BadSignature:
    print("Tampering detected!!")

### 4) Running the Example

In [ ]:
import sys
from django.conf import settings

settings.configure(
    DEBUG=True,
    SECRET_KEY='thisisthesecretkey',
    ROOT_URLCONF=__name__,
    MIDDLEWARE_CLASSES=(
        'django.middleware.common.CommonMiddleware',
        'django.middleware.csrf.CsrfViewMiddleware',
        'django.middleware.clickjacking.XFrameOptionsMiddleware',
    ),
)

from django.conf.urls import url
from django.http import HttpResponse

def index(request):
    return HttpResponse('Hello World')

urlpatterns = (
    url(r'^$', index),
)

if __name__ == "__main__":
    from django.core.management import execute_from_command_line
    execute_from_command_line(sys.argv)

---

## 2. Improvements

### 1) WSGI Application

### 2) Additional Configuration

### 3) Reusable Template